# Daily COVID monitoring data for Santa Barbara county

In [1]:
# Preamble
import os
import IPython
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
# Change working directory (so that Windows task scheduler does not complain)
os.chdir("C:/Users/Alice/Box Sync/PhD/Software/Python/COVID/COVID-SB")

In [3]:
# Request content from URL
html_string = requests.get('https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyDataTable.aspx').content

In [4]:
# Parse HTML
soup = BeautifulSoup(html_string, 'html.parser')

In [5]:
# Extract data for Santa Barbara
table = soup.find_all("tr", class_="ms-rteTableEvenRow-3")
SB = table[21]

In [6]:
# Avg # tests per day (per 100,000 population)
daily_tests = SB.find_all("td", class_="ms-rteTableOddCol-3")[0].text

# Case rate per 100,000
case_rate = SB.find_all("td", class_="ms-rteTableEvenCol-3")[1].text

# Testing positivity (%)
test_positivity = SB.find_all("td", class_="ms-rteTableOddCol-3")[1].text

# % Change in 3-day avg COVID+ hospitalized patients
change_hospit = SB.find_all("td", class_="ms-rteTableEvenCol-3")[2].text

# % ICU beds currently available
ICU_beds = SB.find_all("td", class_="ms-rteTableOddCol-3")[2].text

# % Ventilators currently available
ventilators = SB.find_all("td", class_="ms-rteTableEvenCol-3")[3].text

In [7]:
# Set up Google Sheet API
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

In [8]:
# Append daily data as a new row in Google Sheet
sheet = client.open("COVID Daily Metrics").sheet1
insertRow = [datetime.date.today().strftime("%Y-%m-%d"),
             daily_tests,
             case_rate,
             test_positivity,
             change_hospit,
             ICU_beds,
             ventilators]
sheet.insert_row(insertRow, 2)

{'spreadsheetId': '13GG0FPUFg_5xCouN0ZF3woaPXwjuxL2Dw3uYIfwUW-Q',
 'updatedRange': 'Sheet1!A2:G2',
 'updatedRows': 1,
 'updatedColumns': 7,
 'updatedCells': 7}

In [9]:
# Extract data from Google Sheet as pandas dataframe
sheet = client.open("COVID Daily Metrics").sheet1
data = sheet.get_all_records()
result = pd.DataFrame(data).set_index("Date")

In [10]:
result

,"Daily tests (per 100,000)¹","Case rate (per 100,000)²",Testing positivity (%)³,Change in hospitalizations (%)⁴,ICU beds available (%),Ventilators (available (%)
Date,,,,,,
2020-08-21,208.7,138.5,✔,✔,✔,✔
2020-08-20,214.5,148.8,✔,✔,✔,✔


Notes:
1. 7 day average with a 7-day lag.
2. 14 day average.
3. 7 day average with a 7-day lag.
4. Change in 3 day average.

Source: https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyDataTable.aspx.

Once a county appears on the County Monitoring List for three consecutive days, indoor operations in the sectors listed in Section 3 of the [July 13th State Health Officer Order](https://www.cdph.ca.gov/Programs/CID/DCDC/CDPH%20Document%20Library/COVID-19/SHO%20Order%20Dimming%20Entire%20State%207-13-2020.pdf) must close.

This table tracks where Santa Barbara stands in terms of the criteria for inclusion on the County Monitoring List. As of 20 August 2020, the criteria for inclusion on the monitoring list are:

- (Case rate >100) OR (Case rate >25 AND Positivity >8%)
- Change in hospitalizations >10% increase
- <20% ICU beds available OR <25% ventilators available

These metrics capture three dimensions that the state of California is tracking: Elevated Disease Transmission, Increasing Hospitalization, and Limited Hospital Capacity. If Santa Barbara meets the criteria for removal from the County Monitoring List for a certain metric, then a checkmark ✔ is displayed.

Read more at <https://covid19.ca.gov/roadmap-counties/>.

**See the code to scrape the data: https://github.com/walice/COVID-SB/.**

In [11]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>

In [12]:
# Convert Jupyter Notebook to HTML
! jupyter nbconvert "index.ipynb" --to html --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['remove_cell']" --TagRemovePreprocessor.remove_input_tags="['remove_input']" --no-prompt

[NbConvertApp] Converting notebook index.ipynb to html
[NbConvertApp] Writing 275534 bytes to index.html


In [13]:
# Create daily commit message
now = datetime.datetime.now()
commit_message = "Last scraped on " + str(now)

In [ ]:
# Deploy to GitHub
! git add .
! echo $commit_message > commit_message.txt
! git commit -F commit_message.txt
! git push